# Validaciones con Pydantic

## Creacion de esquemas

Crearemos un esquema de datos que contenga toda la informacion relacionada con una pelicula. 

```py
from pydantic import BaseModel
...
...
class Movie(BaseModel):
    id: int | None = None
    title: str 
    year: int
    rating: float
    category: str
```
Ya tenemos los atributos de la clase Movie, la cual hereda de BaseModel, y contiene todos los atributos que contiene una pelicula. 

### Que significa: id: int | None = None 

*id: int | None = None* es una forma de declarar un parámetro en una función o en una clase:

- id: Esto es el nombre del parámetro. En este caso, se está declarando un parámetro llamado id.

- *int | None*: Esto es una anotación de tipo. En este caso, significa que el parámetro id puede ser de tipo **int** (número entero) o puede ser **None** (es decir, no tener valor). El operador | denota una unión de tipos, lo que indica que el parámetro puede ser de cualquiera de los tipos mencionados.

- *= None*: Esto es un valor por defecto. En este caso, el valor por defecto para id es None. Esto significa que si no se proporciona un valor para id al llamar a la función, se asumirá que su valor es **None**.

Se puede remplazar por:

```py
id: Optional[int] = None
```

Siempre y cuando se importe:

```py
from typing import Optional
```

### Modificando POST

Ojo tiene un bug, que ahora vamos a corregir 🐞

```py
@app.post('/movies', tags=['movies'])
def create_movie(movie: Movie):  👈  
    movies.append(movie)

    return movie
```
- Quiere decir que va requerir una *movie* de tipo *Movie*

#### Corrigiendo el bug.

Al momento de ir al navegador y hacer una peticion, para consultar la pelicula que se acabo de crear:

    http://127.0.0.1:5000/movies/101

Mostraba el siguiente error:

    if item['id'] == id:
    TypeError: 'Movie' object is not subscriptable

Esto es por el tipo de datos como se esta guardando, pues lo esta guardando como tipo *main.Movie*, asi que antes de guardarlo, hay que convertirlo a diccionario.

```py
@app.post('/movies', tags=['movies'])
def create_movie(movie: Movie):    
    movies.append(dict(movie)) 👈

    return movie
```


### Modificando PUT

```py
@app.put('/movies/{id}', tags=['movies'])
def update_movie(id: int, movie: Movie):
    
    for movie in movies:
        if movie['id'] == id:
            
            movie['title'] = movie.title
            movie['year'] = movie.year
            movie['rating'] = movie.rating
            movie['category'] = movie.category
            break
    
    return movie
```

### Verificando Esquema

![](https://i.imgur.com/wS1AJo0.png)

## Validaciones de tipos de datos

Hay cierto tipo de validaciones que ya estan integradas con FastAPI. Por ejemplo vamos a usar el metodo POST y en el body no enviaremos cualquier parametro, por ejemplo el *rating*:

{
  "id": 0,
  "title": "string",
  "year": 0,
  "category": "string"
}

Nos retornara un error *422 Unprocessable Entity*. 

![](https://i.imgur.com/HS108LI.png)

O por ejemplo, al realizar otra peticion con el metodo GET y en ves de enviar un *integer*, enviarmos un texto:

    http://127.0.0.1:5000/movies/a

Y mostrara:

{"detail":[{"type":"int_parsing","loc":["path","id"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"a","url":"https://errors.pydantic.dev/2.4/v/int_parsing"}]}

### Usando la clase Field de pydantic

    from pydantic import BaseModel, Field

#### Modificando la clase Movie

```py
class Movie(BaseModel):
    id: int | None = None
#    id: Optional[int] = None
    title: str = Field(default="movie title", min_length=5, max_length=15)
    year: int = Field(default=2022, le=2022)
    rating: float = Field(ge=0, le=10)
    category: str = Field(default="movie category")
```

- El titulo tenga un minimo de 5 caracteres y un maximo de 15.
- El titulo tenga un valor por defecto
- El año menor o igual al 2022, y por defecto 2022

![](https://i.imgur.com/feKe2st.png)

Tambien es posible dar los valores por defecto, creando una nueva clase *Config* dentro del BaseModel, y dentro de dicha clase:

```py
    class Config:
        schema_extra = {
            "example": {
                "id": 1,
                "title": "Mi película",
                "overview": "Descripción de la película",
                "year": 2022,
                "rating": 9.8,
                "category" : "Acción"
            }
        }

```

Aunque a mi no me funciono. 

## Validaciones de parametros

### Parametros de ruta

Se importa la clase *Path*:

    from fastapi import FastAPI, Body, Path

Y se modifica, para que el parametro de ruta tenga un valor entre 0 y 200:

    def get_movie(id: int = Path(ge=0, le=200)):


Error an intentar colocar 2001 en la 

{"detail":[{"type":"less_than_equal","loc":["path","id"],"msg":"Input should be less than or equal to 200","input":"2001","ctx":{"le":200},"url":"https://errors.pydantic.dev/2.4/v/less_than_equal"}]}

### Parametros Query

Se importa la clase Query:

    from fastapi import FastAPI, Body, Path, Query

Para que el parametro *category* sea minimo de 5 caracteres

    def get_movies_by_category(category: str = Query(min_length=5)):